## <span style="color:#ff5f27;"> 💽 Loading the Data </span>

In [1]:
!pip install -U hopsworks --quiet

In [2]:
!pip install kaggle --quiet

In [3]:
import numpy as np
import pandas as pd
import gdown

In [4]:
# URL from Google Drive shareable link
url = 'https://drive.google.com/uc?id=1LeoFhTNUZiZ9sbn31jZA6DGN3p36Dsvk'

# Filename
output = 'ratebeer.txt'

# Download the file from the URL
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1LeoFhTNUZiZ9sbn31jZA6DGN3p36Dsvk
From (redirected): https://drive.google.com/uc?id=1LeoFhTNUZiZ9sbn31jZA6DGN3p36Dsvk&confirm=t&uuid=7e68d382-637c-4775-a690-8dd4d637c17d
To: /Users/lassehylleberg/Desktop/Data-Engineering-MLOps-Exam-Assignment/Data-Engineering-MLOps-Exam-Assignment/ratebeer.txt
100%|██████████| 1.74G/1.74G [01:46<00:00, 16.3MB/s]


'ratebeer.txt'

In [5]:
# Define the file path to the downloaded file
file_path = 'ratebeer.txt'

# Define columns
columns = ['beer/name', 'beer/beerId', 'beer/brewerId', 'beer/ABV', 'beer/style',
           'review/appearance', 'review/aroma', 'review/palate', 'review/taste',
           'review/overall', 'review/time', 'review/profileName', 'review/text']
data = {col: [] for col in columns}

# Read the file
with open(file_path, "r", encoding="ISO-8859-1") as file:
    current_review = {}
    for line in file:
        line = line.strip()
        if line:
            try:
                key, value = line.split(": ", 1)
                current_review[key] = value
            except ValueError:
                continue
        else:
            if current_review:
                for col in columns:
                    data[col].append(current_review.get(col, np.nan))
                current_review = {}

# Check for any remaining entries to add
if current_review:
    for col in columns:
        data[col].append(current_review.get(col, np.nan))

# Create a DataFrame from the data
df = pd.DataFrame(data)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2924163 entries, 0 to 2924162
Data columns (total 13 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   beer/name           object
 1   beer/beerId         object
 2   beer/brewerId       object
 3   beer/ABV            object
 4   beer/style          object
 5   review/appearance   object
 6   review/aroma        object
 7   review/palate       object
 8   review/taste        object
 9   review/overall      object
 10  review/time         object
 11  review/profileName  object
 12  review/text         object
dtypes: object(13)
memory usage: 290.0+ MB


## <span style="color:#ff5f27;"> 🛠️ Feature Engineering </span>

In [7]:
# Convert numeric columns to float
numeric_cols = ['beer/ABV', 'review/appearance', 'review/aroma', 'review/palate', 'review/taste', 'review/overall']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Convert ID columns to integers, coerce errors which will convert non-convertible values to NaN
id_cols = ['beer/beerId', 'beer/brewerId']
df[id_cols] = df[id_cols].apply(pd.to_numeric, errors='coerce', downcast='integer')

# Convert review time to datetime
df['review/time'] = pd.to_datetime(df['review/time'], errors='coerce')

# Check the changes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2924163 entries, 0 to 2924162
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   beer/name           object        
 1   beer/beerId         float64       
 2   beer/brewerId       int16         
 3   beer/ABV            float64       
 4   beer/style          object        
 5   review/appearance   float64       
 6   review/aroma        float64       
 7   review/palate       float64       
 8   review/taste        float64       
 9   review/overall      float64       
 10  review/time         datetime64[ns]
 11  review/profileName  object        
 12  review/text         object        
dtypes: datetime64[ns](1), float64(7), int16(1), object(4)
memory usage: 273.3+ MB


In [8]:
# Rename columns for Hopsworks compatibility
df.rename(columns={'beer/beerId': 'beer/beerid', 'beer/brewerId': 'beer/brewerid', 'beer/ABV': 'beer/abv', 'review/profileName': 'review/profilename'}, inplace=True)

In [9]:
# Replace / with _ in column names for Hopsworks compatibility
new_columns = {col: col.replace('/', '_').replace('beer/', 'beer_') for col in df.columns}
df.rename(columns=new_columns, inplace=True)

## <span style="color:#ff5f27;"> 🪄 Creating Feature Groups </span>

In [10]:
import hopsworks as hs
project = hs.login()
fs = project.get_feature_store()

/Users/lassehylleberg/anaconda3/envs/MLOPS/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/550037
2024-05-02 11:32:38,755 WARNING: using legacy validation callback


Connected. Call `.close()` to terminate connection gracefully.


In [11]:
print(df.columns)

Index(['beer_name', 'beer_beerid', 'beer_brewerid', 'beer_abv', 'beer_style',
       'review_appearance', 'review_aroma', 'review_palate', 'review_taste',
       'review_overall', 'review_time', 'review_profilename', 'review_text'],
      dtype='object')


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2924163 entries, 0 to 2924162
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   beer_name           object        
 1   beer_beerid         float64       
 2   beer_brewerid       int16         
 3   beer_abv            float64       
 4   beer_style          object        
 5   review_appearance   float64       
 6   review_aroma        float64       
 7   review_palate       float64       
 8   review_taste        float64       
 9   review_overall      float64       
 10  review_time         datetime64[ns]
 11  review_profilename  object        
 12  review_text         object        
dtypes: datetime64[ns](1), float64(7), int16(1), object(4)
memory usage: 273.3+ MB
